In [21]:
import plotly.graph_objs as go
import plotly.offline as pl
import pytz
import scipy.signal as signal
import math
from pandas.tseries.offsets import *
import warnings
import datetime
import pandas as pd
warnings.filterwarnings("ignore")

import arrow
start = arrow.now().shift(days=-1).format('YYYY-MM-DD')
print('昨天的日期：',start)

昨天的日期： 2021-05-19


In [22]:
### 输入的设备列表
devs = {}
devs['868323028604665'] = '中央路工地'  
devs['866154026890382'] = '工大科技楼'
devs['868323028598537'] = '金茂广场'
devs['868323028615547'] = '绿地A工地'
devs['868323028608880'] = '蔡家巷工地'
devs['868323028623152'] = '绿地B工地'
devs['868323028591094'] = '绿地C工地' 
devs['868323028623889'] = '童家山地铁站'
devs['868323028608914'] = '国电南自工地'
devs['868323028615026'] = '江苏省工美馆'
devs['868323028608435'] = '水佐岗'
devs['868323028590914'] = '小市地铁站'

###########################################
devs_zone={}
devs_zone['金茂广场']       = '山西路片区'
devs_zone['中央路331工地']  = '山西路片区'  
devs_zone['国电南自工地']   = '山西路片区'
devs_zone['国电南自工地']    = '山西路片区' 
devs_zone['蔡家巷工地']     = '山西路片区' 
devs_zone['工大丁家桥校区东西科技楼']  = '山西路片区' 
devs_zone['绿地A工地']     = '山西路片区' 
devs_zone['绿地B工地']    = '山西路片区' 
devs_zone['绿地C工地']     = '山西路片区' 
devs_zone['小市地铁站']   =  '山西路片区'
devs_zone['江苏省工美馆']= '清凉山片区'
devs_zone['水佐岗']= '清凉山片区'
import pandas as pd
df_devs_zone = pd.DataFrame.from_dict(devs_zone, orient='index').reset_index().rename(columns={'index':'stn_name',0:'zone_name'})
df_devs_zone

,stn_name,zone_name
0,金茂广场,山西路片区
1,中央路331工地,山西路片区
2,国电南自工地,山西路片区
3,蔡家巷工地,山西路片区
4,工大丁家桥校区东西科技楼,山西路片区
5,绿地A工地,山西路片区
6,绿地B工地,山西路片区
7,绿地C工地,山西路片区
8,小市地铁站,山西路片区
9,江苏省工美馆,清凉山片区


In [23]:
df_empty = pd.read_csv('plotly案例数据.csv',index_col =0)
df_empty

,pm2d5,pm10,tsp,leq,dev,stn_name
2021-05-05 00:01:09+08:00,65.0,85.0,102.0,56.6,8.660000e+14,工大丁家桥校区东西科技楼
2021-05-05 00:02:09+08:00,65.0,86.0,104.0,55.9,8.660000e+14,工大丁家桥校区东西科技楼
2021-05-05 00:03:09+08:00,66.0,86.0,104.0,56.2,8.660000e+14,工大丁家桥校区东西科技楼
2021-05-05 00:04:09+08:00,64.0,85.0,103.0,56.6,8.660000e+14,工大丁家桥校区东西科技楼
2021-05-05 00:05:09+08:00,66.0,86.0,104.0,57.4,8.660000e+14,工大丁家桥校区东西科技楼
...,...,...,...,...,...,...
2021-05-05 23:55:37+08:00,55.0,102.0,129.0,66.5,8.680000e+14,小市地铁站
2021-05-05 23:56:37+08:00,46.0,83.0,106.0,65.8,8.680000e+14,小市地铁站
2021-05-05 23:57:37+08:00,32.0,67.0,84.0,64.7,8.680000e+14,小市地铁站
2021-05-05 23:58:37+08:00,27.0,53.0,66.0,65.0,8.680000e+14,小市地铁站


In [24]:
df_stn_mean = df_empty.groupby(['dev','stn_name'])[['pm2d5', 'pm10', 'tsp', 'leq']].mean().reset_index()
df_stn_mean['day']= start
#df_stn_mean
df_stn_mean['zone_name'] = df_stn_mean.stn_name.apply(lambda x: devs_zone[x])
df_stn_mean.fillna(0,inplace=True)
df_stn_mean['pm2d5'] = df_stn_mean['pm2d5'].astype('int')
df_stn_mean['pm10'] = df_stn_mean['pm10'].astype('int')
df_stn_mean['tsp'] =  df_stn_mean['tsp'].astype('int')
df_stn_mean['leq'] =  df_stn_mean['leq'].astype('int')

df_stn_mean =df_stn_mean.sort_values(by='pm2d5',ascending=True)
df_stn_mean

,dev,stn_name,pm2d5,pm10,tsp,leq,day,zone_name
7,8.680000e+14,绿地C工地,27,55,72,59,2021-05-19,山西路片区
1,8.680000e+14,国电南自工地,33,54,68,61,2021-05-19,山西路片区
2,8.680000e+14,小市地铁站,34,61,77,63,2021-05-19,山西路片区
6,8.680000e+14,绿地B工地,36,63,78,60,2021-05-19,山西路片区
9,8.680000e+14,金茂广场,36,53,64,64,2021-05-19,山西路片区
8,8.680000e+14,蔡家巷工地,38,64,79,58,2021-05-19,山西路片区
4,8.680000e+14,江苏省工美馆,39,67,84,56,2021-05-19,清凉山片区
3,8.680000e+14,水佐岗,40,69,86,57,2021-05-19,清凉山片区
5,8.680000e+14,绿地A工地,40,63,78,55,2021-05-19,山西路片区
0,8.660000e+14,工大丁家桥校区东西科技楼,44,64,75,67,2021-05-19,山西路片区


In [25]:
rank_kind ='average'
df_stn_mean['group_sort_tsp']    =df_stn_mean.groupby(['zone_name'])['tsp'].rank(ascending=1,method= rank_kind)
df_stn_mean['group_sort_tsp'] =df_stn_mean['group_sort_tsp'].astype(int)
df_stn_mean['group_sort_pm2d5']  =df_stn_mean.groupby(['zone_name'])['pm2d5'].rank(ascending=1,method= rank_kind)
df_stn_mean['group_sort_pm2d5'] =df_stn_mean['group_sort_pm2d5'].astype(int)
df_stn_mean['group_sort_leq']    =df_stn_mean.groupby(['zone_name'])['leq'].rank(ascending=1,method= rank_kind)
df_stn_mean['group_sort_leq'] = df_stn_mean['group_sort_leq'] .astype(int)
df_stn_mean['group_sort_pm10']   =df_stn_mean.groupby(['zone_name'])['pm10'].rank(ascending=1,method= rank_kind)
df_stn_mean['group_sort_pm10']=df_stn_mean['group_sort_pm10'].astype(int)
#df_stn_mean.columns
result = df_stn_mean[[ 'zone_name','stn_name','tsp','group_sort_tsp', 'pm2d5','group_sort_pm2d5' 
             , 'pm10', 'group_sort_pm10',
              'dev','day']].sort_values(by='tsp',ascending=True)

result

,zone_name,stn_name,tsp,group_sort_tsp,pm2d5,group_sort_pm2d5,pm10,group_sort_pm10,dev,day
9,山西路片区,金茂广场,64,1,36,4,53,1,8.680000e+14,2021-05-19
1,山西路片区,国电南自工地,68,2,33,2,54,2,8.680000e+14,2021-05-19
7,山西路片区,绿地C工地,72,3,27,1,55,3,8.680000e+14,2021-05-19
0,山西路片区,工大丁家桥校区东西科技楼,75,4,44,8,64,7,8.660000e+14,2021-05-19
2,山西路片区,小市地铁站,77,5,34,3,61,4,8.680000e+14,2021-05-19
6,山西路片区,绿地B工地,78,6,36,4,63,5,8.680000e+14,2021-05-19
5,山西路片区,绿地A工地,78,6,40,7,63,5,8.680000e+14,2021-05-19
8,山西路片区,蔡家巷工地,79,8,38,6,64,7,8.680000e+14,2021-05-19
4,清凉山片区,江苏省工美馆,84,1,39,1,67,1,8.680000e+14,2021-05-19
3,清凉山片区,水佐岗,86,2,40,2,69,2,8.680000e+14,2021-05-19


In [26]:
#result
RESULT =result[result.zone_name=='山西路片区'].append(result[result.zone_name=='清凉山片区'])\
.append(result[result.zone_name=='地铁7号线'])

#df_devs_zone
R =df_devs_zone.merge(RESULT,how= 'outer',on=['stn_name','zone_name']).sort_values(by='tsp',ascending=True)

#R
#R
#R[(R.zone_name=='山西片区')&(R.leq.isna() )]
#R[(R.zone_name=='清凉山片区')&(pd.isnull(R['leq']) )]

#pd.isnull(df_empty['pm2d5'])
R_nan=R[pd.isnull(R['pm2d5'])]

#R_nan
#result[np.isnan(result['leq'])]

In [27]:
R

,stn_name,zone_name,tsp,group_sort_tsp,pm2d5,group_sort_pm2d5,pm10,group_sort_pm10,dev,day
0,金茂广场,山西路片区,64.0,1.0,36.0,4.0,53.0,1.0,8.680000e+14,2021-05-19
2,国电南自工地,山西路片区,68.0,2.0,33.0,2.0,54.0,2.0,8.680000e+14,2021-05-19
7,绿地C工地,山西路片区,72.0,3.0,27.0,1.0,55.0,3.0,8.680000e+14,2021-05-19
4,工大丁家桥校区东西科技楼,山西路片区,75.0,4.0,44.0,8.0,64.0,7.0,8.660000e+14,2021-05-19
8,小市地铁站,山西路片区,77.0,5.0,34.0,3.0,61.0,4.0,8.680000e+14,2021-05-19
5,绿地A工地,山西路片区,78.0,6.0,40.0,7.0,63.0,5.0,8.680000e+14,2021-05-19
6,绿地B工地,山西路片区,78.0,6.0,36.0,4.0,63.0,5.0,8.680000e+14,2021-05-19
3,蔡家巷工地,山西路片区,79.0,8.0,38.0,6.0,64.0,7.0,8.680000e+14,2021-05-19
9,江苏省工美馆,清凉山片区,84.0,1.0,39.0,1.0,67.0,1.0,8.680000e+14,2021-05-19
10,水佐岗,清凉山片区,86.0,2.0,40.0,2.0,69.0,2.0,8.680000e+14,2021-05-19


# 发现 中央路331工地缺失

把各个片区的数据缺失的工地 所在行放到所在片区的最后

In [28]:
RR =R[(R.zone_name=='山西路片区')&(R.pm2d5.notna() )].append(R_nan[R_nan.zone_name=='山西路片区']).\
append(R[(R.zone_name=='清凉山片区')&(R.pm2d5.notna() )]).append(R_nan[R_nan.zone_name=='清凉山片区']).\
append(R[(R.zone_name=='地铁7号线')&(R.pm2d5.notna() )]).append(R_nan[R_nan.zone_name=='地铁7号线'])
RR

,stn_name,zone_name,tsp,group_sort_tsp,pm2d5,group_sort_pm2d5,pm10,group_sort_pm10,dev,day
0,金茂广场,山西路片区,64.0,1.0,36.0,4.0,53.0,1.0,8.680000e+14,2021-05-19
2,国电南自工地,山西路片区,68.0,2.0,33.0,2.0,54.0,2.0,8.680000e+14,2021-05-19
7,绿地C工地,山西路片区,72.0,3.0,27.0,1.0,55.0,3.0,8.680000e+14,2021-05-19
4,工大丁家桥校区东西科技楼,山西路片区,75.0,4.0,44.0,8.0,64.0,7.0,8.660000e+14,2021-05-19
8,小市地铁站,山西路片区,77.0,5.0,34.0,3.0,61.0,4.0,8.680000e+14,2021-05-19
5,绿地A工地,山西路片区,78.0,6.0,40.0,7.0,63.0,5.0,8.680000e+14,2021-05-19
6,绿地B工地,山西路片区,78.0,6.0,36.0,4.0,63.0,5.0,8.680000e+14,2021-05-19
3,蔡家巷工地,山西路片区,79.0,8.0,38.0,6.0,64.0,7.0,8.680000e+14,2021-05-19
1,中央路331工地,山西路片区,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,江苏省工美馆,清凉山片区,84.0,1.0,39.0,1.0,67.0,1.0,8.680000e+14,2021-05-19


## 填充缺失值 为 \

In [29]:
RR[['zone_name', 'stn_name','tsp','group_sort_tsp', 'pm10', 
    'group_sort_pm10', 'pm2d5', 'group_sort_pm2d5', 'dev', 'day']].fillna('\\').replace(999999,'\\')

,zone_name,stn_name,tsp,group_sort_tsp,pm10,group_sort_pm10,pm2d5,group_sort_pm2d5,dev,day
0,山西路片区,金茂广场,64,1,53,1,36,4,8.68e+14,2021-05-19
2,山西路片区,国电南自工地,68,2,54,2,33,2,8.68e+14,2021-05-19
7,山西路片区,绿地C工地,72,3,55,3,27,1,8.68e+14,2021-05-19
4,山西路片区,工大丁家桥校区东西科技楼,75,4,64,7,44,8,8.66e+14,2021-05-19
8,山西路片区,小市地铁站,77,5,61,4,34,3,8.68e+14,2021-05-19
5,山西路片区,绿地A工地,78,6,63,5,40,7,8.68e+14,2021-05-19
6,山西路片区,绿地B工地,78,6,63,5,36,4,8.68e+14,2021-05-19
3,山西路片区,蔡家巷工地,79,8,64,7,38,6,8.68e+14,2021-05-19
1,山西路片区,中央路331工地,\,\,\,\,\,\,\,\
9,清凉山片区,江苏省工美馆,84,1,67,1,39,1,8.68e+14,2021-05-19


# 将默认的字符串索引 设置为 时间索引，方便使用resample

In [30]:
df_empty.index = pd.DatetimeIndex(df_empty.index)

In [31]:
df_empty.index

DatetimeIndex(['2021-05-05 00:01:09+08:00', '2021-05-05 00:02:09+08:00',
               '2021-05-05 00:03:09+08:00', '2021-05-05 00:04:09+08:00',
               '2021-05-05 00:05:09+08:00', '2021-05-05 00:06:09+08:00',
               '2021-05-05 00:07:09+08:00', '2021-05-05 00:08:09+08:00',
               '2021-05-05 00:09:09+08:00', '2021-05-05 00:10:09+08:00',
               ...
               '2021-05-05 23:50:37+08:00', '2021-05-05 23:51:37+08:00',
               '2021-05-05 23:52:37+08:00', '2021-05-05 23:53:37+08:00',
               '2021-05-05 23:54:37+08:00', '2021-05-05 23:55:37+08:00',
               '2021-05-05 23:56:37+08:00', '2021-05-05 23:57:37+08:00',
               '2021-05-05 23:58:37+08:00', '2021-05-05 23:59:37+08:00'],
              dtype='datetime64[ns, pytz.FixedOffset(480)]', length=14108, freq=None)

In [32]:
df_hour =df_empty.groupby('stn_name').resample('1h',label = 'right').mean().reset_index().rename(columns={'level_1':'day_hour'})
df_hour

,stn_name,day_hour,pm2d5,pm10,tsp,leq,dev
0,国电南自工地,2021-05-05 01:00:00+08:00,58.134426,80.829508,102.473770,52.991803,8.680000e+14
1,国电南自工地,2021-05-05 02:00:00+08:00,59.970492,83.059016,105.344262,52.808197,8.680000e+14
2,国电南自工地,2021-05-05 03:00:00+08:00,57.414754,80.470492,102.059016,52.277049,8.680000e+14
3,国电南自工地,2021-05-05 04:00:00+08:00,57.800000,80.822951,102.432787,52.563934,8.680000e+14
4,国电南自工地,2021-05-05 05:00:00+08:00,62.650820,86.008197,109.178689,51.449180,8.680000e+14
...,...,...,...,...,...,...,...
235,金茂广场,2021-05-05 20:00:00+08:00,25.775410,41.229508,48.627869,63.206557,8.680000e+14
236,金茂广场,2021-05-05 21:00:00+08:00,24.916393,39.626230,46.496721,59.988525,8.680000e+14
237,金茂广场,2021-05-05 22:00:00+08:00,22.549180,36.418033,42.270492,58.909836,8.680000e+14
238,金茂广场,2021-05-05 23:00:00+08:00,24.559016,38.857377,45.500000,58.373770,8.680000e+14


# 求山西路 小时值

In [33]:
df_hour['day_hour'] = df_hour['day_hour'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
#df_hour
df_hour['zone_name'] = df_hour.stn_name.apply(lambda x: devs_zone[x])
df_hour_shanxi = df_hour[df_hour['zone_name']=='山西路片区']
df_hour_shanxi

,stn_name,day_hour,pm2d5,pm10,tsp,leq,dev,zone_name
0,国电南自工地,2021-05-05 01:00:00,58.134426,80.829508,102.473770,52.991803,8.680000e+14,山西路片区
1,国电南自工地,2021-05-05 02:00:00,59.970492,83.059016,105.344262,52.808197,8.680000e+14,山西路片区
2,国电南自工地,2021-05-05 03:00:00,57.414754,80.470492,102.059016,52.277049,8.680000e+14,山西路片区
3,国电南自工地,2021-05-05 04:00:00,57.800000,80.822951,102.432787,52.563934,8.680000e+14,山西路片区
4,国电南自工地,2021-05-05 05:00:00,62.650820,86.008197,109.178689,51.449180,8.680000e+14,山西路片区
...,...,...,...,...,...,...,...,...
235,金茂广场,2021-05-05 20:00:00,25.775410,41.229508,48.627869,63.206557,8.680000e+14,山西路片区
236,金茂广场,2021-05-05 21:00:00,24.916393,39.626230,46.496721,59.988525,8.680000e+14,山西路片区
237,金茂广场,2021-05-05 22:00:00,22.549180,36.418033,42.270492,58.909836,8.680000e+14,山西路片区
238,金茂广场,2021-05-05 23:00:00,24.559016,38.857377,45.500000,58.373770,8.680000e+14,山西路片区


## 赋值颜色

In [34]:
colors ={'中央路331工地': '#4a5256',
 '国电南自工地': '#0000FF',
 '小市地铁站': '#9932CD ',
 '工大丁家桥校区东西科技楼': '#A67D3D',
 '绿地A工地': '#FFFF00 ',
 '绿地B工地': '#00FFFF',
 '绿地C工地': '#99CC32',
 '蔡家巷工地': '#70DB93',
 '金茂广场': '#FF0000'}

In [35]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=4, cols=1,
                    subplot_titles=("日均值及其排名","山西路PM2.5小时值","山西路PM10小时值","山西路TSP小时值" ),
                    vertical_spacing=0.070,
            specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}],
            [{"type": "scatter"}]
                  ])


stn_list = list(df_hour_shanxi.stn_name.unique())

In [36]:
fig.add_trace(
    go.Table(
        header=dict(
            values=["站点名称", "区域<br>名称", "TSP",
                    "TSP区域排名", "PM2.5", "PM2.5<br>区域排名",
                    "PM10", "PM10分组排名"],
            font=dict(size=15),
            align="center"
        ),
        cells=dict(
            values=[RR[k].tolist() for k in RR.columns[0:8]],
            align = "center")
    ),  row=1, col=1
   
)


for stn in stn_list:
    fig.add_trace(go.Scatter(x=df_hour_shanxi[df_hour['stn_name']==stn]['day_hour'].tolist(), 
                             y=df_hour_shanxi[df_hour_shanxi['stn_name']==stn]['pm2d5'].tolist(),
                             name=stn,legendgroup=stn ,line = dict(   color = colors[stn], width = 1.5  ),showlegend=False),
             
                  
                  row=2, col=1 )
    
for stn in stn_list:
    fig.add_trace(go.Scatter(x=df_hour_shanxi[df_hour['stn_name']==stn]['day_hour'].tolist(), 
                             y=df_hour_shanxi[df_hour_shanxi['stn_name']==stn]['pm10'].tolist(),
                             name=stn,legendgroup=stn,line = dict(   color = colors[stn], width = 1.5  ),showlegend=False),
             
                  
                  row=3, col=1 )
    
for stn in stn_list:
    fig.add_trace(go.Scatter(x=df_hour_shanxi[df_hour['stn_name']==stn]['day_hour'].tolist(), 
                             y=df_hour_shanxi[df_hour_shanxi['stn_name']==stn]['tsp'].tolist(),
                             name=stn,legendgroup=stn,line = dict(   color = colors[stn], width = 1.5  ),showlegend=True),
                  row=4, col=1 )


layout = go.Layout(
    xaxis=dict(title=start +'', zeroline=False, rangeslider=dict(visible=True)),
    yaxis=dict(title=' ', zeroline=False),title='山西路各工地',showlegend=True,
    )    
    
        
    
fig.update_layout(height=1400, width=1400, title_text="",#plot_bgcolor='rgba(1,0,0,0)' ,
                 legend=dict(
        
        title_font_family="Times New Roman",
        font=dict(family="Courier", size=14,color="black" ),
        bgcolor="LightSteelBlue",
                     
        bordercolor="Black", borderwidth=2,
        orientation="h",
        yanchor="bottom",
        y=0.76,
        xanchor="right",
        x=0.95     )
            )    


pl.plot(fig, filename=start+'_multiple-subplots')
#fig.show

'2021-05-19_multiple-subplots.html'

In [37]:
fig.show

<bound method BaseFigure.show of Figure({
    'data': [{'cells': {'align': 'center',
                        'values': [['金茂广场', '国电南自工地', '绿地C工地', '工大丁家桥校区东西科技楼',
                                   '小市地铁站', '绿地A工地', '绿地B工地', '蔡家巷工地', '中央路331工地',
                                   '江苏省工美馆', '水佐岗'], ['山西路片区', '山西路片区', '山西路片区',
                                   '山西路片区', '山西路片区', '山西路片区', '山西路片区', '山西路片区',
                                   '山西路片区', '清凉山片区', '清凉山片区'], [64.0, 68.0, 72.0,
                                   75.0, 77.0, 78.0, 78.0, 79.0, nan, 84.0, 86.0],
                                   [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 6.0, 8.0, nan,
                                   1.0, 2.0], [36.0, 33.0, 27.0, 44.0, 34.0, 40.0,
                                   36.0, 38.0, nan, 39.0, 40.0], [4.0, 2.0, 1.0,
                                   8.0, 3.0, 7.0, 4.0, 6.0, nan, 1.0, 2.0], [53.0,
                                   54.0, 55.0, 64.0, 61.0, 63.0, 63.0, 64.0, nan,
                 

limit_output extension: Maximum message size of 10000 exceeded with 40914 characters